<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reviewa y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

# Modelo de ML con Regresión Logística.

### Target = Promedios de Estrellas

### Librerías a Utilizar:

In [180]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import json
import gcsfs
import ast
from textblob import TextBlob   #0.17.1
import pandas as pd
import folium
import webbrowser
import textwrap
from folium.plugins import MarkerCluster

### Cargando Dataset Consolidado Final para ML: 'Consolidado_Businees_Review_Tip.parquet'

In [203]:
### Cargando DataSets Consolidado para Análisis de ML: 'Consolidado_Businees_Review_Tip.parquet'
# Ruta completa en Cloud Storage
#ruta_eda_datalike = "gs://gmy/Datasets_ML/df_ML_promedios_Final.parquet"
ruta_eda_datalike = "gs://gmy/Datasets_ML/NuevosDatasetsML/Consolidado_Businees_Review_Tip.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_ML = pd.read_parquet(ruta_eda_datalike, storage_options={"project": "Proyecto Final - Henry"})
df_ML


,business_id,name,address,city,state,latitude,longitude,stars,review_count,Categorias,...,T_S_P_Review,T_S_Neutros_Review,T_S_N_Review,Total_useful,Total_funny,Total_cool,Total_Sentimientos_Tip,T_S_P_Tip,T_S_Neutros_Tip,T_S_N_Tip
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,39.955505,-75.155564,4.0,80,Restaurants,...,82,0,5,131,47,95,10,0,0,0
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,40.338183,-75.471659,4.5,13,Food,...,12,0,1,2,0,0,1,0,0,0
2,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,36.269593,-87.058943,2.0,6,Food,...,3,1,2,0,0,0,2,0,0,0
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,MO,36.208102,-86.768170,1.5,10,Restaurants,...,7,0,3,2,2,0,2,0,0,0
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,MO,27.955269,-82.456320,4.0,10,Restaurants,...,10,0,1,29,5,17,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44631,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,43.615401,-116.284689,4.0,33,Restaurants,...,29,1,4,34,14,18,9,0,0,0
44632,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,PA,32.409552,-110.943073,4.5,41,Food,...,41,0,2,86,5,20,36,0,0,0
44633,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,MO,39.935982,-75.158665,4.5,35,Restaurants,...,34,0,1,24,9,9,4,0,0,0
44634,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,PA,39.856185,-75.427725,4.5,14,Restaurants,...,12,0,2,21,5,15,1,0,0,0


In [208]:
df = df_ML.copy()

In [209]:
### Definiendo Columnas de Inversión
Columnas_inversion = ['name', 'city', 'state', 
                 'stars', 'review_count', 'Categorias', 'SubCategorias',
                 'Total_CincoEstrellas', 'Total_Estrellas', 
                 'T_S_P_Review',
                 'Total_useful', 'Total_funny', 'Total_cool', 
                 'T_S_P_Tip',
                 'latitude', 'longitude','address']

In [210]:
df_inversion = df[Columnas_inversion]

In [214]:
### Filtrando por 'Categorias' igual a 'Restaurants' o 'Food'
filtro_categorias = filtro_categorias[(filtro_categorias['Categorias'] == 'Restaurants') | (df['Categorias'] == 'Food')]

### Ordenando el DataFrame por las siguientes columnas especificadas:
columnas_orden = ['Total_CincoEstrellas', 'review_count', 'T_S_P_Review', 'T_S_P_Tip', 'Total_useful', 'Total_funny', 'Total_cool']
resultado = filtro_categorias.sort_values(by=columnas_orden, ascending=False)

df_mapa = resultado[:20].copy()
resultado

/var/tmp/ipykernel_26860/3459718003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtro_categorias = filtro_categorias[(filtro_categorias['Categorias'] == 'Restaurants') | (df['Categorias'] == 'Food')]


,name,city,state,stars,review_count,Categorias,SubCategorias,Total_CincoEstrellas,Total_Estrellas,T_S_P_Review,Total_useful,Total_funny,Total_cool,T_S_P_Tip,latitude,longitude,address
33393,Oceana Grill,New Orleans,FL,4.0,7400,Restaurants,"Seafood, Cajun/Creole",4012,7516,7050,3692,1206,1906,0,29.956231,-90.067563,739 Conti St
27291,Hattie B’s Hot Chicken - Nashville,Nashville,NJ,4.5,6093,Restaurants,"Chicken Shop, Southern",3838,6160,5429,3756,1831,2845,0,36.151387,-86.796603,112 19th Ave S
33760,Acme Oyster House,New Orleans,AZ,4.0,7568,Restaurants,"Seafood, Live/Raw Food",3645,7673,7143,4431,1938,3017,0,29.954273,-90.068965,724 Iberville St
9414,Commander's Palace,New Orleans,PA,4.5,4876,Restaurants,"French, Cocktail Bars",3095,4969,4753,5320,2005,3046,0,29.928735,-90.084196,1403 Washington Ave
43654,Ruby Slipper - New Orleans,New Orleans,FL,4.5,5193,Restaurants,"American (Traditional), American (New)",3075,5264,4911,2189,705,1386,0,29.951025,-90.067394,200 Magazine St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35204,Old World Tomato Pie,Florence Township,IN,1.0,5,Restaurants,"Event Planning & Services, Pizza",0,5,1,0,0,0,0,40.119600,-74.805800,401 Broad St
22968,A&W,Edmonton,IN,1.0,5,Restaurants,"Fast Food, Burgers",0,5,0,13,0,1,0,53.595967,-113.492507,13440 97th Street
42485,KFC,Edmonton,IN,1.0,5,Restaurants,"Fast Food, American (Traditional)",0,5,0,11,0,0,0,53.509187,-113.675652,2474 Guardian Road NW
23743,Edible Arrangements,Largo,FL,1.0,5,Food,"Shopping, Candy Stores",0,5,0,8,0,0,0,27.892934,-82.784805,"10500 Ulmerton Rd, Ste 660"


In [215]:
### Mapa que Muestras las primeras 20 mejores opciones de inversión
df = df_mapa

map = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], 
                zoom_start=7, control_scale=True)

marker_cluster = MarkerCluster().add_to(map)

title = 'Proyecto Yelp & Google Maps - Reseñas y Recomendaciones </br> Proyecto Final Grupo Nº 7 </br>@utores: Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García'
title_html = '<h3 align="center" style="font-size:16px"><b>{}</b></h3>'.format(title)

#Loop through each row in the dataframe
for i,row in df.iterrows():
            
    # Formatting the text1
    text1 = f"Name: <strong>{row['name']}</strong></br> \
    City: <strong>{row['city']}</strong></br> \
    Address: <strong>{row['address']}</strong></br> \
    State: <strong>{row['state']}</strong></br>  \
    Categorias: <strong>{row['Categorias']}</strong></br> \
    SubCategorias: <strong>{row['SubCategorias']}</strong></br> \
    Total_Estrellas: <strong>{row['Total_Estrellas']}</strong></br> \
    Total_CincoEstrellas: <strong>{row['Total_CincoEstrellas']}</strong></br> \
    T_S_P_Review: <strong>{row['T_S_P_Review']}</strong></br> \
    Total_useful: <strong>{row['Total_useful']}</strong></br> \
    Total_funny: <strong>{row['Total_funny']}</strong></br> \
    Total_cool: <strong>{row['Total_cool']}</strong></br> \
    T_S_P_Tip: <strong>{row['T_S_P_Tip']}</strong></br> \
    </br> "

    # Formatting the text2 = ATRIBUTOS
    text2 = f"Name: <strong>{row['name']}</strong></br>\
    City: <strong>{row['city']}</strong></br> \
    Address: <strong>{row['address']}</strong></br> \
    State: <strong>{row['state']}</strong></br> \
    latitude: <strong>{row['latitude']}</strong></br> \
    longitude: <strong>{row['longitude']}</strong></br> \
    </br>"
        
    #Initialise the popup using the iframe
    text2 = folium.Popup(text2, min_width=300, max_width=450)  # Size Text2    OC_FechaCierreIncidente
    
    
    folium.Marker(location=[row['latitude'],row['longitude']],
                popup = text2, tooltip =  text1,
                icon=folium.Icon( prefix='fa') ).add_to(marker_cluster)  

map.get_root().html.add_child(folium.Element(title_html))
#map.save("MapaPF_Grupo07.html")    
map

### Respaldando archivo en formato 'PARQUET'

In [179]:
### Respaldando archivo en formato 'PARQUET'
Ubicacion = "gs://gmy/Datasets_ML/NuevosDatasetsML"
NombreDelArchivo = 'Consolidado_Businees_Review_Tip_Normalizado' # Consolidado Limpio con UNOS y CEROS
Extension = '.parquet'
file = os.path.join(Ubicacion, NombreDelArchivo) + Extension
df.to_parquet(file)

### Fin

### Para Llevar a Streamlit

In [ ]:
### Cargando DataSets Consolidado para Análisis de ML: 'Consolidado_Businees_Review_Tip.parquet'
# Ruta completa en Cloud Storage
#ruta_eda_datalike = "gs://gmy/Datasets_ML/df_ML_promedios_Final.parquet"
ruta_eda_datalike = "gs://gmy/Datasets_ML/NuevosDatasetsML/Consolidado_Businees_Review_Tip.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_ML = pd.read_parquet(ruta_eda_datalike, storage_options={"project": "Proyecto Final - Henry"})


### Defiiendo Columnas de Inversion
Columnas_inversion = ['name', 'city', 'state', 
                 'stars', 'review_count', 'Categorias', 'SubCategorias',
                 'Total_Estrellas', 'Total_CincoEstrellas' , 
                 'T_S_P_Review',
                 'Total_useful', 'Total_funny', 'Total_cool', 
                 'T_S_P_Tip',
                 'latitude', 'longitude','address']

### Creando DataSets de Inversión
df_inversion = df[Columnas_inversion]

### Aplicando Filtro par anuevo dataset de Inversion
filtro_categorias = df_inversion[(df_inversion['Categorias'] == 'Restaurants') | (df['Categorias'] == 'Food')]

### Filtrando por 'Categorias' igual a 'Restaurants' o 'Food'
filtro_categorias = filtro_categorias[(filtro_categorias['Categorias'] == 'Restaurants') | (df['Categorias'] == 'Food')]

### Ordenando el DataFrame por las siguientes columnas especificadas:
columnas_orden = ['Total_CincoEstrellas', 'review_count', 'T_S_P_Review', 'T_S_P_Tip', 'Total_useful', 'Total_funny', 'Total_cool']
resultado = filtro_categorias.sort_values(by=columnas_orden, ascending=False)

### Imprime los mejores 10 resultados de Negocios para Invertir
resultado[:10]

### Muestra el Mapa de los 20 primeros Negocios de Inversion
df_mapa = resultado[:20].copy()


### Inicio del Mapa
map = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], 
                zoom_start=7, control_scale=True)

marker_cluster = MarkerCluster().add_to(map)

title = 'Proyecto Yelp & Google Maps - Reseñas y Recomendaciones </br> Proyecto Final Grupo Nº 7 </br>@utores: Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García'
title_html = '<h3 align="center" style="font-size:16px"><b>{}</b></h3>'.format(title)

#Loop through each row in the dataframe
for i,row in df.iterrows():
            
    # Formatting the text1
    text1 = f"Name: <strong>{row['name']}</strong></br> \
    City: <strong>{row['city']}</strong></br> \
    Address: <strong>{row['address']}</strong></br> \
    State: <strong>{row['state']}</strong></br>  \
    Categorias: <strong>{row['Categorias']}</strong></br> \
    SubCategorias: <strong>{row['SubCategorias']}</strong></br> \
    Total_Estrellas: <strong>{row['Total_Estrellas']}</strong></br> \
    Total_CincoEstrellas: <strong>{row['Total_CincoEstrellas']}</strong></br> \
    T_S_P_Review: <strong>{row['T_S_P_Review']}</strong></br> \
    Total_useful: <strong>{row['Total_useful']}</strong></br> \
    Total_funny: <strong>{row['Total_funny']}</strong></br> \
    Total_cool: <strong>{row['Total_cool']}</strong></br> \
    T_S_P_Tip: <strong>{row['T_S_P_Tip']}</strong></br> \
    </br> "

    # Formatting the text2 = ATRIBUTOS
    text2 = f"Name: <strong>{row['name']}</strong></br>\
    City: <strong>{row['city']}</strong></br> \
    Address: <strong>{row['address']}</strong></br> \
    State: <strong>{row['state']}</strong></br> \
    latitude: <strong>{row['latitude']}</strong></br> \
    longitude: <strong>{row['longitude']}</strong></br> \
    </br>"
        
    #Initialise the popup using the iframe
    text2 = folium.Popup(text2, min_width=300, max_width=450)  # Size Text2    OC_FechaCierreIncidente
    
    
    folium.Marker(location=[row['latitude'],row['longitude']],
                popup = text2, tooltip =  text1,
                icon=folium.Icon( prefix='fa') ).add_to(marker_cluster)  

map.get_root().html.add_child(folium.Element(title_html))
#map.save("MapaPF_Grupo07.html")    
map

### FIN


